In [ ]:
%reload_ext autoreload
%autoreload 2

import math, os, sys
import json
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.offline as py
from pathlib import Path
from datetime import datetime
from scipy import ndimage


from ipywidgets import interactive, HBox, VBox
HOME = os.path.expanduser("~")
PATH = os.path.join(HOME, 'programming/pipeline_utility')
sys.path.append(PATH)

from utilities.file_location import FileLocationManager
from utilities.sqlcontroller import SqlController

In [ ]:
animal = 'MD589'
sqlController = SqlController(animal)
atlas_dir = Path('/net/birdstore/Active_Atlas_Data/data_root/atlas_data/atlasV7')
origin_dir = atlas_dir / 'origin'
volume_dir = atlas_dir / 'structure'

In [ ]:
for origin_file, volume_file in zip(sorted(origin_dir.iterdir()), sorted(volume_dir.iterdir())):
    assert origin_file.stem == volume_file.stem
    name = origin_file.stem
    origin = np.loadtxt(origin_file)
    if name.endswith('_L'):
        abbreviation = name.replace('_L','')
        side = 'L'
    elif name.endswith('_R'):
        abbreviation = name.replace('_R','')
        side = 'R'
    else:
        abbreviation = name
        side = 'S'
    if abbreviation == 'RMC':
        continue
    #structure = sqlController.get_structure(abbreviation)
    
    volume = np.load(volume_file)
    volume = np.rot90(volume, axes=(0, 1))
    volume = np.flip(volume, axis=0)

    # computer volume center of mass in raw array coordinates
    center = (origin + ndimage.measurements.center_of_mass(volume))
    x,y,z = center
    
    structure = sqlController.get_structure(abbreviation)
    print("insert into center_of_mass (prep_id, structure_id, x,y,section,side,created) values (",end="" )
    print(f"'Atlas','{structure.id}',  {x},{y},{z}, '{side}', NOW());")
    #sqlController.add_center_of_mass(abbreviation, 'Atlas', x, y, z, side)



In [ ]:
abbreviation, animal, x,y,z,side = 'RtTg', 'Atlas', -121.54587226705019, 62.24769427642217, -3.499999999999986, 'S'
sqlController.add_center_of_mass(abbreviation, animal, x, y, z, side)

In [ ]:
def find_values(id, json_repr):
    results = []
    def _decode_dict(a_dict):
        try:
            results.append(a_dict[id])
        except KeyError:
            pass
        return a_dict

    json.loads(json_repr, object_hook=_decode_dict) # Return value ignored.
    return results